In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

PROJECT_ROOT = '/content/drive/MyDrive/Colab Notebooks/soai'   # 변경 시 여기만 수정
import sys, os
sys.path.append(PROJECT_ROOT)                  # agent.py 임포트를 위해

Mounted at /content/drive


In [8]:
!pip install -qU langchain langchain-community \
                langchain-google-genai \
                sentence-transformers transformers accelerate
!pip install -qU langgraph

In [9]:
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "fantasy-npc-project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [10]:
# (A) Gemini – 가장 단순
from langchain_google_genai import ChatGoogleGenerativeAI
import os

api_key = userdata.get('GEMINI_API_KEY')
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             google_api_key=api_key,
                             temperature=0.7)

# (B) Qwen-3 4B (GPU 여유 있을 때)
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_community.llms import HuggingFacePipeline
# model_id = "Qwen/Qwen-3-4B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# model     = AutoModelForCausalLM.from_pretrained(
#                 model_id, device_map="auto", trust_remote_code=True)
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
#                 max_new_tokens=1024, temperature=0.7, do_sample=True)
# llm = HuggingFacePipeline(pipeline=pipe)

In [11]:
# import importlib, sys

# agent.py 파일 수정한 경우 아래 코드 실행하기
# 수정한 내용을 반영하려면 ↓ 두 줄이면 충분합니다.
# importlib.invalidate_caches()          # 파이썬 모듈 캐시 초기화
# if "agent" in sys.modules:             # 이미 로드돼 있었다면
#     importlib.reload(sys.modules["agent"])
# else:
#     import agent                       # 처음 불러오는 경우


In [12]:
from agent import build_langgraph          # agent.py 안의 함수

graph1 = build_langgraph(
    file_path   = os.path.join(PROJECT_ROOT, "asuna"),
    analysis_llm=llm,
    response_llm=llm,
    summary_llm =llm,                      # 생략하면 response_llm 재사용
    me="asuna",
    you="rocky"
)

graph2 = build_langgraph(
    file_path   = os.path.join(PROJECT_ROOT, "rocky"),
    analysis_llm=llm,
    response_llm=llm,
    summary_llm =llm,                      # 생략하면 response_llm 재사용
    me="rocky",
    you="asuna"
)

In [13]:
state = {"question": "오늘 퀘스트는 뭐가 있는지 확인해볼까?"}

for i in range(50):
    asuna_ans = graph1.invoke(state)
    print("asuna: ", asuna_ans["response"])
    state = {"question": asuna_ans["response"]}
    rocky_ans = graph2.invoke(state)
    print("rocky: ", rocky_ans["response"])
    state = {"question": rocky_ans["response"]}

asuna:  퀘스트 목록 확인은 시스템 창을 열면 바로 알 수 있는 정보야. 하지만 네가 그걸 묻는 걸 보면, 단순히 목록을 확인하는 것 이상의 의미를 원하는 것 같네. 현재 진행 가능한 퀘스트는 세 가지. '엘프 여왕의 눈물', '저주받은 숲의 정화', 그리고 '잃어버린 왕국의 유산'. 중요도는 '엘프 여왕의 눈물'이 가장 높아. 숲의 균형을 되찾는 것과 직결되니까. 난이도는... 네 레벨이라면 '저주받은 숲의 정화'가 조금 버거울 수도 있겠어. 하지만 키리토와 함께라면 불가능하진 않겠지. 아니면, 나와 함께 가겠어?
rocky:  흐음, 역시 눈썰미는 좋네. 단순히 퀘스트 목록을 확인하려는 건 아니었어. '엘프 여왕의 눈물'은... 뭐, 스토리 자체는 뻔하지만, 보상이 꽤 괜찮을 거야. 난이도는 중간 정도? 혼자서는 좀 버거울 거고. '저주받은 숲의 정화'는 꽤 까다로운 퀘스트인데, 독 저항이 높은 사람이랑 같이 가는 게 좋을 거야. 아스나, 네가 가면 딱이겠는데? 마지막으로 '잃어버린 왕국의 유산'은... 이건 난이도가 좀 높아. 게다가 함정이나 퍼즐이 많을 것 같아. 머리 쓰는 건 질색이라... 키리토, 네가 좀 도와줘야 할 것 같은데? 물론, 내가 먼저 함정을 파악하고 정보를 넘겨줄 수도 있고. 어떤 퀘스트부터 시작할지는 네 선택에 달렸어. 중요한 건, 왜 그걸 선택했는지 나한테 설명해줘야 한다는 거지. 안 그러면 심심하잖아?
asuna:  로키, 네가 뭘 선택할지 궁금하네. 굳이 묻는 걸 보면, 나한테 결정을 맡기고 싶은 건 아닌 것 같고... '엘프 여왕의 눈물'이 시급한 건 맞지만, 네 의견도 존중할게. 이유를 들어보고 함께 결정하는 게 좋겠지. 이번엔 네가 하고 싶은 걸 먼저 해보는 것도 괜찮을 것 같아.
rocky:  엘프 여왕의 눈물을 얻는 게 급한 건 알겠는데... 굳이 내 의견을 물어본 걸 보면, 뭔가 다른 이유가 있는 거겠지? 솔직히 말해서, 셋 다 딱히 끌리는 퀘스트는 없어. 하지만 굳이 하나를 고르라면... '저주받은 숲의 정화'

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 11
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 9
}
]